In [ ]:
import numpy as np
import pickle

In [5]:
# Parameters
SEQ_LEN = 50   # sequence length for training
note_sequences_file = "D:\Study\Ai_Music_Composer\project\data/note_sequence.pkl"
output_file = "D:\Study\Ai_Music_Composer\project\data\generator_training_data.pkl"

In [6]:
# Load extracted note sequences with moods
with open(note_sequences_file, "rb") as f:
    note_sequences = pickle.load(f)

print(f"Loaded {len(note_sequences)} sequences")

Loaded 4000 sequences


In [7]:
# Build a note vocabulary
all_notes = [note for seq in note_sequences for note in seq["notes"]]
unique_notes = sorted(list(set(all_notes)))
note_to_int = {note: number for number, note in enumerate(unique_notes)}
int_to_note = {number: note for note, number in note_to_int.items()}

print(f"Unique notes: {len(unique_notes)}")

Unique notes: 128


In [8]:
# Encode moods into integers
unique_moods = sorted(list(set(seq["mood"] for seq in note_sequences)))
mood_to_int = {mood: idx for idx, mood in enumerate(unique_moods)}
int_to_mood = {idx: mood for mood, idx in mood_to_int.items()}

print(f"Moods: {mood_to_int}")

Moods: {'angry': 0, 'happy': 1, 'relaxed': 2, 'sad': 3}


In [9]:
# Prepare training sequences
network_input = []
network_output = []
network_mood = []

for seq in note_sequences:
    notes = seq["notes"]
    mood = seq["mood"]
    if len(notes) <= SEQ_LEN:
        continue
    
    # Convert notes to integers
    encoded_notes = [note_to_int[n] for n in notes]
    
    for i in range(len(encoded_notes) - SEQ_LEN):
        seq_in = encoded_notes[i:i + SEQ_LEN]
        seq_out = encoded_notes[i + SEQ_LEN]
        
        network_input.append(seq_in)
        network_output.append(seq_out)
        network_mood.append(mood_to_int[mood])

print(f"Total training sequences: {len(network_input)}")

Total training sequences: 7872357


In [10]:
# Convert to numpy arrays
network_input = np.array(network_input)
network_output = np.array(network_output)
network_mood = np.array(network_mood)

In [11]:
# Save everything
with open(output_file, "wb") as f:
    pickle.dump({
        "input": network_input,
        "output": network_output,
        "mood": network_mood,
        "note_to_int": note_to_int,
        "int_to_note": int_to_note,
        "mood_to_int": mood_to_int,
        "int_to_mood": int_to_mood,
        "seq_len": SEQ_LEN
    }, f)

print(f"Saved training data to {output_file} ✅")

Saved training data to D:\Study\Ai_Music_Composer\project\data\generator_training_data.pkl ✅
